# [Queues*, Stacks](https://stackoverflow.com/questions/10974922/what-is-the-basic-difference-between-stack-and-queue), [Trees](https://en.wikipedia.org/wiki/Tree_(data_structure)#Terminology_used_in_trees) and Graphs

*but without queues.

## `Node`

Das propriedades básicas de um _node_ n:

- n.STATE
- n.PATH-COST
- n.PARENT
- n.ACTION

In [460]:
class Node:
    def __init__ (self, state, cost, parent=None, action=None):
        self.state  = state
        self.cost   = cost
        self.parent = parent
        self.action = action
        
        if self.parent:
            self.depth = self.parent.depth + 1
        else:
            self.depth = 0
    
    def __repr__ (self):
        return '<Node {}>'.format(self.state)

## `Stack`

In [461]:
class Stack:
    def __init__ (self):
        self.items = []
    
    def push(self, item):
        self.items.append(item)
    
    def pop(self):
        return self.items.pop()
    
    def peek(self):
        return self.items[len(self.items)-1]
    
    def __len__(self):
        return len(self.items)

## `Problem`

Das propriedades de um problema bem definido:

- estado inicial
- possíveis ações
- consequências das ações
- teste de solução
- função de custo de caminho

In [462]:
class Problem:
    """
       essa  classe depende  do  problema  estudado,
       mas em geral todos os problemas compartilham
       a estrutura abaixo.
    """
    
    def __init__ (self):
        self.states = '...'
        self.costs  = '...'
        self.goal   = '...'
    
    
    def start(self):
        return '...'
    
    def is_state(self, state):
        return state in self.states
    
    def actions(self, state):
        if state in self.states:
            return self.states(state)
        else:
            return None
    
    def next_state(self, state, action):
        if action in self.actions(state):
            return action
        else:
            return None
    
    def is_goal_state(self, state):
        return state == self.goal
    
    def cost(self, state, action):
        return self.costs(state, action)

# Problema do Viajante ... no Metrô em SP

Detalhes de como as coordenadas foram obtidas em "[Dados](#dados)", no final do notebook.

In [463]:
import os, json, unidecode

import numpy as np
import pandas as pd
import geopandas, folium

from IPython.display import IFrame, display

In [464]:
#with open('data/stations.json', 'r') as f:
with open('data/metrosp_stations_v3.json', 'r') as f:
    j = json.load(f)

data = json.dumps(j)

df   = pd.read_json(data)
df.reset_index(level=0, inplace=True)
df.drop(labels=['index'], axis=1, inplace=True) 

In [465]:
colors = { 'azul'    : 'darkblue',
           'verde'   : 'green',
           'vermelha': 'red',
           'amarela' : 'beige',
           'lilas'   : 'purple',
           'prata'   : 'lightgray'}

df.loc[:, 'cor'] = '.'
df['cor'] = df['line'].apply( lambda x: colors[sorted(x)[0]] if len(x) == 1 else 'black')

In [466]:
df.sample(5)

,lat,line,lon,name,neigh,station,cor
62,-23.588826,[prata],-46.544725,Sao Lucas,"[oratorio, camilo-haddad]",sao-lucas,lightgray
31,-23.598305,[lilas],-46.645577,Hospital Sao Paulo,"[aacd-servidor, santa-cruz]",hospital-sao-paulo,purple
54,-23.598938,"[azul, lilas]",-46.638867,Santa Cruz,"[vila-mariana, praca-da-arvore, hospital-sao-p...",santa-cruz,black
70,-23.480069,[azul],-46.603241,Tucuruvi,[parada-inglesa],tucuruvi,darkblue
5,-23.547825,[vermelha],-46.639180,Anhangabau,"[se, republica]",anhangabau,red


In [467]:
centroid = (df['lat'].mean(), df['lon'].mean())

sp = folium.Map( location=(centroid[0], centroid[1]), zoom_start=12 )
for i, r in df.iterrows():
    folium.Marker( location = [r['lat'], r['lon']],
                   icon     = folium.Icon(color=r['cor']),
                   popup    = r['name'] ).add_to(sp)
sp

Convertendo em estrutura gráfica:

In [468]:
GRAPH = { r['station'] : {'neigh': r['neigh'], 'pos': (r['lat'], r['lon'])} for i,r in df.iterrows() }

In [469]:
def costFunc(s, S):
    sy, sx = s['pos'][0], s['pos'][1]
    Sy, Sx = S['pos'][0], S['pos'][1]
    return ((sx-Sx)**2 + (sy-Sy)**2)**(0.5)

COSTS = {}
for s in GRAPH:
    for S in GRAPH[s]['neigh']:
        COSTS[(s, S)] = costFunc(GRAPH[s], GRAPH[S])

- - -

Finalmente, definindo o problema:

In [470]:
class Problem(object):
    """
       essa  classe depende  do  problema  estudado,
       mas em geral todos os problemas compartilham
       a estrutura abaixo.
    """
    
    def __init__ (self, s0='luz', s='luz'):
        self.states = GRAPH
        self.costs  = COSTS
        self.goal   = s
        self.start  = s0
    
    
    def start(self):
        return self.start
    
    def is_state(self, state):
        return state in self.states
    
    def actions(self, state):
        if state in self.states:
            return self.states[state]['neigh']
        else:
            return None
    
    def next_state(self, state, action):
        if action in self.actions(state):
            return action
        else:
            return None
    
    def is_goal_state(self, state):
        return state == self.goal
    
    def cost(self, state, action):
        return self.costs[(state, action)]

## Depth-first Search

In [471]:
def depthFirstSearch(problem):
    node     = Node(problem.start, 0)
    frontier = Stack()
    frontier.push(node)
    explored = set()
    while len(frontier) > 0:
        node = frontier.pop()
        explored.add( node.state )
        
        if problem.is_goal_state( node.state ):
            return node
        
        for act in problem.actions( node.state ):
            next_state = problem.next_state( node.state, act )
            if next_state not in explored:
                cost = problem.cost( node.state, act ) + node.cost
                frontier.push( Node(next_state, cost, node, act) )
    return None


## Teste

In [472]:
p = Problem('butanta', 'corinthians-itaquera')
a = depthFirstSearch(p)

In [473]:
def ans(problem, sol):
    sy, sx = GRAPH[problem.start]['pos'][0], GRAPH[problem.start]['pos'][1]
    gy, gx = GRAPH[problem.goal]['pos'][0], GRAPH[problem.goal]['pos'][1]
    
    centroid = (np.mean([sy,gy]), np.mean([sx,gx]))

    map = folium.Map( location=(centroid[0], centroid[1]), zoom_start=12 )
    for i, r in df.iterrows():
        folium.Marker( location = [r['lat'], r['lon']],
                   icon     = folium.Icon(color=r['cor']),
                   popup    = r['name'] ).add_to(map)
    
    parent = sol
    points = [(gy, gx), (GRAPH[parent.state]['pos'][0], GRAPH[parent.state]['pos'][1])]
    folium.CircleMarker(location=points[-1], radius='4').add_to(map)
    while parent != None:
        points.append( (GRAPH[parent.state]['pos'][0], GRAPH[parent.state]['pos'][1]) )
        folium.CircleMarker(location=points[-1], radius='4').add_to(map)
        parent = parent.parent
    
    folium.PolyLine(locations=points, color="black", weight=4).add_to(map)
    
    display(map)

ans(p, a)

# Referências

1. Russell, S., Norvig, P. & Davis, E. (2010). **Artificial intelligence : a modern approach**. Upper Saddle River, NJ: Prentice Hall.

- - -

# Dados

Acrescentando estações à base de estações usada inicialmente:

In [106]:
from geopy.geocoders import Nominatim

nom = Nominatim(user_agent='metrosp')

S = {}
L = {'azul'     : """Tucuruvi • Parada Inglesa • Jardim São Paulo-Ayrton Senna • Santana • Carandiru • Portuguesa-Tietê • Armênia • Tiradentes • Luz • São Bento • Sé • Japão-Liberdade • São Joaquim • Vergueiro • Paraíso • Ana Rosa • Vila Mariana • Santa Cruz • Praça Da Árvore • Saúde • São Judas • Conceição • Jabaquara""".split('•'),
     'verde'    : """Vila Prudente • Tamanduateí • Sacomã • Alto Do Ipiranga • Santos-Imigrantes • Chácara Klabin • Ana Rosa • Paraíso • Brigadeiro • Trianon-Masp • Consolação • Clínicas • Santuário Nossa Senhora De Fátima-Sumaré • Vila Madalena""".split('•'), 
     'vermelha' : """Palmeiras-Barra Funda • Marechal Deodoro • Santa Cecília • República • Anhangabaú • Sé • Pedro II • Brás • Bresser-Mooca • Belém • Tatuapé • Carrão • Penha • Vila Matilde • Guilhermina-Esperança • Patriarca-Vila Ré • Artur Alvim • Corinthians-Itaquera""".split('•'),
     'prata'    : """Vila União • Vila Tolstói • Camilo Haddad • São Lucas • Oratório • Vila Prudente""".split('•'),
     'amarela'  : """São Paulo-Morumbi • Butantã • Pinheiros • Faria Lima • Fradique Coutinho • Oscar Freire • Paulista • Higienópolis-Mackenzie • República • Luz""".split('•'),
     'lilas'    : """Chácara Klabin • Santa Cruz • Hospital São Paulo • Aacd-Servidor • Moema • Eucaliptos • Brooklin • Borba Gato • Alto Da Boa Vista • Adolfo Pinheiro • Largo Treze • Santo Amaro • Giovanni Gronchi • Vila Das Belezas • Campo Limpo • Capão Redondo""".split('•')
    }

In [107]:
for line in sorted(L):
    for station in L[line]:
        _D   = {}
        NAME = unidecode.unidecode( station ).strip().replace('-', ' ')
        TAG  = NAME.lower().strip().replace(' ', '-')

        try:
            if TAG not in S:
                loc = nom.geocode('Estacão São Paulo {}'.format( NAME ))
            
                _D = {'station': TAG,
                  'name'   : NAME, 
                  'lat'    : loc.latitude,
                  'lon'    : loc.longitude,
                  'line'   : line
                 }
                print(_D)
                S[TAG] = _D
        except:
            _D = {'station': TAG,
                  'name'   : NAME, 
                  'lat'    : 0,
                  'lon'    : 0,
                  'line'   : line
                 }
            print(_D)
            S[TAG] = _D

{'name': 'Sao Paulo Morumbi', 'station': 'sao-paulo-morumbi', 'lat': -23.5861368, 'lon': -46.7239121, 'line': 'amarela'}
{'name': 'Butanta', 'station': 'butanta', 'lat': -23.5718995, 'lon': -46.7080896, 'line': 'amarela'}
{'name': 'Pinheiros', 'station': 'pinheiros', 'lat': -23.567249, 'lon': -46.7019515, 'line': 'amarela'}
{'name': 'Faria Lima', 'station': 'faria-lima', 'lat': -23.5672551, 'lon': -46.6939585, 'line': 'amarela'}
{'name': 'Fradique Coutinho', 'station': 'fradique-coutinho', 'lat': -23.5662284, 'lon': -46.6841386, 'line': 'amarela'}
{'name': 'Oscar Freire', 'station': 'oscar-freire', 'lat': -23.5607503, 'lon': -46.6720286, 'line': 'amarela'}
{'name': 'Paulista', 'station': 'paulista', 'lat': -23.5552461, 'lon': -46.6622682, 'line': 'amarela'}
{'name': 'Higienopolis Mackenzie', 'station': 'higienopolis-mackenzie', 'lat': -23.5489592, 'lon': -46.6523049, 'line': 'amarela'}
{'name': 'Republica', 'station': 'republica', 'lat': -23.5440945, 'lon': -46.642665, 'line': 'amarela

In [ ]:
# tentando preencher missings

df_stations = pd.DataFrame.from_dict(S, orient='index')
out = df_stations[df_stations['lat']==0].index.tolist()

for line in sorted(L):
    for station in L[line]:
        _D   = {}
        NAME = unidecode.unidecode( station ).strip().replace('-', ' ')
        TAG  = NAME.lower().strip().replace(' ', '-')

        try:
            if TAG in out:
                loc = nom.geocode('Estacão Metrô {}'.format( station ))
            
                _D = {'station': TAG,
                  'name'   : NAME, 
                  'lat'    : loc.latitude,
                  'lon'    : loc.longitude,
                  'line'   : line
                 }
                print(_D)
                S[TAG] = _D
        except:
            _D = {'station': TAG,
                  'name'   : NAME, 
                  'lat'    : 0,
                  'lon'    : 0,
                  'line'   : line
                 }
            print(_D)
            S[TAG] = _D

In [165]:
# preenchimento manual de missings remanescentes

df_stations.at['hospital-sao-paulo', 'lat'] = -23.5983051
df_stations.at['hospital-sao-paulo', 'lon'] = -46.6455771

df_stations.at['jardim-sao-paulo-ayrton-senna', 'lat'] = -23.4922867
df_stations.at['jardim-sao-paulo-ayrton-senna', 'lon'] = -46.6188575

df_stations.at['patriarca-vila-re', 'lat'] = -23.5311445
df_stations.at['patriarca-vila-re', 'lon'] = -46.5036382

df_stations.at['santuario-nossa-senhora-de-fatima-sumare', 'lat'] = -23.5507519
df_stations.at['santuario-nossa-senhora-de-fatima-sumare', 'lon'] = -46.6800962

df_stations.at['clinicas', 'lat'] = -23.5541125
df_stations.at['clinicas', 'lon'] = -46.6730048

df_stations.at['santa-cruz', 'lat'] = -23.5989383
df_stations.at['santa-cruz', 'lon'] = -46.6388672

df_stations.at['se', 'lat'] = -23.5500992
df_stations.at['se', 'lon'] = -46.635515

df_stations.at['sao-bento', 'lat'] = -23.5443802
df_stations.at['sao-bento', 'lon'] = -46.6363175

# https://www.google.com/maps/place/Hospital+S%C3%A3o+Paulo/@-23.5983051,-46.6455771,15z/data=!4m12!1m6!3m5!1s0x0:0x4cb1766d6b35a30c!2sHospital+S%C3%A3o+Paulo!8m2!3d-23.5983051!4d-46.6455771!3m4!1s0x0:0x4cb1766d6b35a30c!8m2!3d-23.5983051!4d-46.6455771
# https://www.google.com/maps/place/Jardim+S%C3%A3o+Paulo-Ayrton+Senna/@-23.4922867,-46.6188575,17z/data=!4m12!1m6!3m5!1s0x94cef63c891f8393:0x3a7f712ed770bb4!2sJardim+S%C3%A3o+Paulo-Ayrton+Senna!8m2!3d-23.4922867!4d-46.6166635!3m4!1s0x94cef63c891f8393:0x3a7f712ed770bb4!8m2!3d-23.4922867!4d-46.6166635
# https://www.google.com/maps/place/Patriarca/@-23.5311445,-46.5036382,17z/data=!4m12!1m6!3m5!1s0x94ce60ba0b7fced7:0x62da75682bf9997f!2sPatriarca!8m2!3d-23.5311445!4d-46.5014442!3m4!1s0x94ce60ba0b7fced7:0x62da75682bf9997f!8m2!3d-23.5311445!4d-46.5014442
# https://www.google.com/maps/place/Santu%C3%A1rio+N.Sra.+de+F%C3%A1tima+-+Sumar%C3%A9/@-23.5507519,-46.6800962,17z/data=!4m12!1m6!3m5!1s0x94ce578ef82bb161:0xc2abad00ecdf5009!2sSantu%C3%A1rio+N.Sra.+de+F%C3%A1tima+-+Sumar%C3%A9!8m2!3d-23.5507519!4d-46.6779022!3m4!1s0x94ce578ef82bb161:0xc2abad00ecdf5009!8m2!3d-23.5507519!4d-46.6779022
# https://www.google.com.br/maps/place/Cl%C3%ADnicas/@-23.5541125,-46.6730048,17z/data=!3m1!4b1!4m5!3m4!1s0x94ce57876320e9e9:0xb5a6e6174c111b8d!8m2!3d-23.5541125!4d-46.6708108
# https://www.google.com.br/maps/place/Sta.Cruz/@-23.5989383,-46.6388672,17z/data=!3m1!4b1!4m5!3m4!1s0x94ce5a33d4d887c5:0x1561cee36c26c0a5!8m2!3d-23.5989383!4d-46.6366732
# https://www.google.com.br/maps/place/S%C3%A9/@-23.5500992,-46.635515,17z/data=!3m1!4b1!4m5!3m4!1s0x94ce59aa5b004689:0x37c720ec525c8bd9!8m2!3d-23.5500992!4d-46.633321
# https://www.google.com.br/maps/place/S%C3%A3o+Bento/@-23.5443802,-46.6363175,17z/data=!3m1!4b1!4m5!3m4!1s0x94ce5854e29aabab:0x345f955246d87385!8m2!3d-23.5443802!4d-46.6341235

In [185]:
INTEGRACAO = dict()

for k in sorted(L):
    for s in sorted(L[k]):
        if s.strip() in INTEGRACAO:
            INTEGRACAO[s.strip()].append(k)
        else:
            INTEGRACAO[s.strip()] = [k]

for k in INTEGRACAO:
    NAME = unidecode.unidecode( k ).strip().replace('-', ' ')
    TAG  = NAME.lower().strip().replace(' ', '-')
    df_stations.at[TAG,'line'] = INTEGRACAO[k]

buscando conexões:

In [ ]:
def d(a,b):
    ay, ax = a['lat'], a['lon']
    by, bx = b['lat'], b['lon']
    return ( (ax-bx)**2 + (ay-by)**2 )**(0.5)

PONTAS = {'sao-paulo-morumbi', 'tucuruvi', 'capao-redondo'}

MINDIST = {}

for line in sorted(L):
    for station in sorted(L[line]):
        NAME = unidecode.unidecode( station ).strip().replace('-', ' ')
        TAG  = NAME.lower().strip().replace(' ', '-')
        S = df_stations.loc[TAG]
        D = {}
        
        for i,r in df_stations.iterrows():
            D[i] = d(S, r)
        
        D = pd.DataFrame.from_dict(D, orient='index')
        D.sort_values(by=0, ascending=True, inplace=True)
        
        print(station, D[1:4], len(df_stations.loc[TAG]['line']))
        break

In [290]:
def dist(a, b, O=[0, 0]):
    Oy, Ox = O[0], O[1]
    ay, ax = Oy-a['lat'], Ox-a['lon']
    by, bx = Oy-b['lat'], Ox-b['lon']
    
    return ( (ax-bx)**2 + (ay-by)**2 )**(0.5)

PONTAS  = { 'azul'   : 'tucuruvi',
           'verde'   : 'vila-madalena',
           'vermelha': 'corinthians-itaquera',
           'amarela' : 'sao-paulo-morumbi',
           'lilas'   : 'capao-redondo',
           'prata'   : 'vila-prudente'}
DISTANCES = {}

for line in sorted(L):
    ROOT = PONTAS[line]
    C = df_stations.loc[ROOT]
    D = {}
    for station in sorted(L[line]):
        NAME = unidecode.unidecode( station ).strip().replace('-', ' ')
        TAG  = NAME.lower().strip().replace(' ', '-')
        D[TAG] = dist(C, df_stations.loc[TAG])
        
    DF = pd.DataFrame.from_dict(D, orient='index')
    DF.sort_values(by=0, ascending=True, inplace=True)
    DISTANCES[line] = DF

In [299]:
GRAPH = {}

for line in sorted(L):
    _df = DISTANCES[line].copy()
    _df.reset_index(inplace=True)
    
    for station in sorted(L[line]):
        NAME = unidecode.unidecode( station ).strip().replace('-', ' ')
        TAG  = NAME.lower().strip().replace(' ', '-')
        #
        _index    = _df.loc[_df['index'] == TAG].index[0]

        _index_m1 = _index - 1
        _index_p1 = _index + 1
        
        if _index_m1 < 0: 
            _index_m1 = None
        if _index_p1 > len(_df)-1: 
            _index_p1 = None
        
        for i in [_index_m1, _index_p1]:
            if i != None:
                if TAG in GRAPH:
                    GRAPH[ TAG ].append( _df.iloc[ i ]['index'] )
                else:
                    GRAPH[ TAG ] = [ _df.iloc[ i ]['index'] ]

In [313]:
df_stations.loc[:, 'neigh'] = '.'
df_stations['neigh']        = df_stations['station'].apply( lambda x: GRAPH[str(x)] )

In [314]:
df_stations.to_csv('data//metrosp_stations_v3.csv')
df_stations.to_json('data/metrosp_stations_v3.json')

- - - 

tentativa frustrada:

In [ ]:
# import bs4, time, unidecode
# from selenium import webdriver

# L = {'azul'     : """Tucuruvi • Parada Inglesa • Jardim São Paulo-Ayrton Senna • Santana • Carandiru • Portuguesa-Tietê • Armênia • Tiradentes • Luz • São Bento • Sé • Japão-Liberdade • São Joaquim • Vergueiro • Paraíso • Ana Rosa • Vila Mariana • Santa Cruz • Praça Da Árvore • Saúde • São Judas • Conceição • Jabaquara""".split('•'),
#      'verde'    : """Vila Prudente • Tamanduateí • Sacomã • Alto Do Ipiranga • Santos-Imigrantes • Chácara Klabin • Ana Rosa • Paraíso • Brigadeiro • Trianon-Masp • Consolação • Clínicas • Santuário Nossa Senhora De Fátima-Sumaré • Vila Madalena""".split('•'), 
#      'vermelha' : """Palmeiras-Barra Funda • Marechal Deodoro • Santa Cecília • República • Anhangabaú • Sé • Pedro II • Brás • Bresser-Mooca • Belém • Tatuapé • Carrão • Penha • Vila Matilde • Guilhermina-Esperança • Patriarca-Vila Ré • Artur Alvim • Corinthians-Itaquera""".split('•'),
#      'prata'    : """Vila União • Vila Tolstói • Camilo Haddad • São Lucas • Oratório • Vila Prudente""".split('•'),
#      'amarela'  : """São Paulo-Morumbi • Butantã • Pinheiros • Faria Lima • Fradique Coutinho • Oscar Freire • Paulista • Higienópolis-Mackenzie • República • Luz""".split('•'),
#      'lilas'    : """Chácara Klabin • Santa Cruz • Hospital São Paulo • Aacd-Servidor • Moema • Eucaliptos • Brooklin • Borba Gato • Alto Da Boa Vista • Adolfo Pinheiro • Largo Treze • Santo Amaro • Giovanni Gronchi • Vila Das Belezas • Campo Limpo • Capão Redondo""".split('•')
#     }

# driver = webdriver.Chrome("/usr/lib/chromium-browser/chromedriver")

# for line in sorted(L):
#     for station in L[line]:
#         print(station)
#         FLAG = False
#         _D   = {}
#         NAME = unidecode.unidecode( station ).strip().replace('-', ' ')
#         TAG  = NAME.lower().strip().replace(' ', '-')

        
#         driver.get('https://www.duckduckgo.com/?q={}'.format( 'São Paulo Metrô {} "latitude" site:www.adistanciaentre.com'.format( NAME ) ))
#         soup = BS(driver.page_source, 'lxml').select('div.g a')
#         time.sleep(10)
        

#         for url in [i['href'] for i in soup]:
#             if FLAG == False:
#                 if 'LatLongHistoria' in url and :
#                     html = requests.get(url).text
#                     soup = BS(html)
                    
#                     _D = {'station': TAG,
#                           'name'   : NAME, 
#                           'lat'    : float( soup.find('span', id='MC_GMD_lblLatitude').get_text().replace(',', '.') ),
#                           'lon'    : float( soup.find('span', id='MC_GMD_lblLongitude').get_text().replace(',', '.') ),
#                           'line'   : line
#                          }
#                     FLAG = True
#         if len(_D) > 0:
#             S[TAG] = _D
#         else:
#             S[TAG] = {'station': TAG,
#                       'name'   : NAME, 
#                       'lat'    : 0,
#                       'lon'    : 0,
#                       'line'   : line
#                          }
# driver.quit()